In [ ]:
result_files = [
    'models_4/FPN_224_f0_84d1a7.23.results.npy',
    'models_4/FPN_224_f1_f5d73b.24.results.npy',
    'models_4/FPN_224_f2_0970dc.24.results.npy',
    'models_4/FPN_224_f3_efc38c.22.results.npy'
    
]
thresholds = [ 0.5, 0.46, 0.63, 0.51 ]
submission_file_name = 'models_4/FPN_224_f0123'


In [ ]:
import numpy as np
import pandas as pd
import load_data

In [ ]:
for i, f in enumerate(result_files):
    r = np.load(f)
    print(r.shape)
    if i == 0:
        s = r
    else:
        s += r
    print(s.shape, np.sum(s))
s /= len(result_files)
    

In [ ]:
threshold_best = np.mean(thresholds)
threshold_best

In [ ]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
test_df = load_data.LoadData(train_data = False, DEV_MODE_RANGE = 0)

In [ ]:
import time
t1 = time.time()
pred_dict = {idx: rle_encode((s[i] > threshold_best)) for i, idx in enumerate(test_df.index.values)}
t2 = time.time()


In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv(submission_file_name+'.submission.csv.gz', compression = 'gzip')